In [ ]:
pip install -q hvplot

In [ ]:
pip install -q holoviews

In [ ]:
%env HV_DOC_HTML = True


In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
hvplot.extension("bokeh")
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaledData = StandardScaler().fit_transform(df_market_data)

In [ ]:
# Create a DataFrame with the scaled data
marketDataDF = pd.DataFrame(
    scaledData,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
marketDataDF["coin_id"]= df_market_data.index
# Set the coinid column as index

marketDataDF =marketDataDF.set_index("coin_id")
# Display sample data
marketDataDF.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
kValues = list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values
inertiaValues=[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for k in kValues:
  kModel = KMeans(n_clusters=k, random_state=0, n_init='auto')
  kModel.fit(marketDataDF)
  inertiaValues.append(kModel.inertia_)

inertiaValues[:5]

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbowData = {
    "k": kValues,
    "inertia": inertiaValues
}

# Create a DataFrame with the data to plot the Elbow curve
elbowDF = pd.DataFrame(elbowData)
elbowDF.head()

In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
hvplot.extension("bokeh")
elbowPlot= elbowDF.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=kValues
)
elbowPlot

#### Answer the following question:

**Question:** What is the best value for `k`?

**Answer:**

The best value for "k" is 4.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
kModel =KMeans(n_clusters=4, n_init='auto')

In [ ]:
# Fit the K-Means model using the scaled data
kModel.fit(marketDataDF)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
cryptoPredictions = kModel.predict(marketDataDF)

# Print the resulting array of cluster values.
cryptoPredictions

In [ ]:
# Create a copy of the DataFrame
marketDataPredictionsDF= marketDataDF.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
marketDataPredictionsDF["cryptoCluster"] = cryptoPredictions

# Display sample data
marketDataPredictionsDF.head(3)

In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
hvplot.extension("bokeh")

clusterPlot = marketDataPredictionsDF.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by= "cryptoCluster",
    hover_cols=["coin_id"],
    marker=["hex", "square","cross","triangle"],
    title= "Crypto Clusters"
)
clusterPlot

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pcaModel= PCA(n_components=3)


In [ ]:
# Use the PCA model with `fit_transform` to reduce to
# three principal components.
cryptoPCAData = pcaModel.fit_transform(marketDataDF)
# View the first five rows of the DataFrame.
cryptoPCAData[:5]

In [ ]:
pd.DataFrame(cryptoPCAData).head()

In [ ]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pcaModel.explained_variance_ratio_

In [ ]:
#dump of all features that contributed to variance ratios
pd.DataFrame(
    pcaModel.components_,
    columns= marketDataDF.columns,
    index=["PC1","PC2","PC3"]
    )

#### Answer the following question:

**Question:** What is the total explained variance of the three principal components?

**Answer:** ~89.5%

In [ ]:
# Create a new DataFrame with the PCA data.
pcaDF = pd.DataFrame(
    cryptoPCAData,
    columns=["PC1", "PC2","PC3"]
)
# Creating a DataFrame with the PCA data

# Copy the crypto names from the original data
pcaDF["coin_id"]= df_market_data.index

# Set the coinid column as index
pcaDF= pcaDF.set_index("coin_id")

# Display sample data
pcaDF.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
kValues = list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values

inertiaValues=[]
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in kValues:
  kModel = KMeans(n_clusters=k, random_state=0, n_init='auto')
  kModel.fit(pcaDF)
  inertiaValues.append(kModel.inertia_)

inertiaValues[:5]

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
pcaElbow={
    "k": kValues,
    "inertia": inertiaValues
}
# Create a DataFrame with the data to plot the Elbow curve
pcaElbowDF= pd.DataFrame(pcaElbow)
pcaElbowDF.head()

In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
hvplot.extension("bokeh")
pcaElbowPlot = pcaElbowDF.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve Using PCA Data",
    xticks=kValues
)
pcaElbowPlot

#### Answer the following questions:

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**
The best value is still '4'

* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:**
  Not really, both values are "4"

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
pcaKModel = KMeans(n_clusters=4, n_init='auto')

In [ ]:
# Fit the K-Means model using the PCA data
pcaKModel.fit(pcaDF)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pcaPredictions = pcaKModel.predict(pcaDF)
# Print the resulting array of cluster values.
pcaPredictions

In [ ]:
# Create a copy of the DataFrame with the PCA data
pcaPredictionsDF= pcaDF.copy()

# Add a new column to the DataFrame with the predicted clusters

pcaPredictionsDF["PCA Predicted Cluster"]= pcaPredictions
# Display sample data
pcaPredictionsDF.head()

In [ ]:
print(pcaPredictionsDF.columns)


In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
hvplot.extension("bokeh")
clusterPCAPlot = pcaPredictionsDF.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="PCA Predicted Cluster",
    hover_cols=["coin_id"],
    marker=["hex", "square","cross","circle"],
    title="Crypto Clusters Using PCA"
)
clusterPCAPlot

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow
hvplot.extension("bokeh")
elbowPlot + pcaElbowPlot


In [ ]:
# Composite plot to contrast the clusters
hvplot.extension("bokeh")
clusterPlot + clusterPCAPlot

#### Answer the following question:

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Less features can be used to get a similar like performance on the predictive models since 4 clusters are identified.
  